In [1]:
textgrid_dir = "/Users/joregan/Playing/hsi_ctmedit/textgrid"
wav_dir = "/Users/joregan/Playing/hsi/audio/"

In [2]:
from pydub import AudioSegment

In [3]:
from pathlib import Path

textgrid_path = Path(textgrid_dir)
wav_path = Path(wav_dir)

In [4]:
from praatio import textgrid

In [21]:
import re

def noise_event(text):
    m = re.match("^\[([^\]]+)\]$", text)
    return (m is not None)

In [5]:
test_audio = AudioSegment.from_file(wav_path / "hsi_6_0718_209_003_inter.wav")

In [9]:
red = test_audio.set_frame_rate(16000)

In [11]:
tg = textgrid.openTextgrid(textgrid_path / "hsi_7_0719_211_004_inter.TextGrid", False)

In [18]:
# tg._tierDict["utterances"].entries
entries = {}

tier = tg.getTier("utterances")
for entry in tier.entries:
    text = entry.label

    

(Interval(start=0.4819935034938646, end=4.070102841824428, label='Okay, so we can do the landlord stuff.'),
 Interval(start=5.8815966619896445, end=8.660968384458382, label='So yeah, thanks for renting me out this place.'),
 Interval(start=22.42615370788416, end=25.564154039703702, label="It's a bit small for me, I want a bigger apartment."),
 Interval(start=30.993257898679108, end=32.16505713264647, label='Yes of course.'),
 Interval(start=46.180879624851336, end=47.07745114822835, label='No.'),
 Interval(start=52.1457772986572, end=53.53120956350062, label='Ah I think it was like that before.'),
 Interval(start=57.247836407248656, end=57.58322056969709, label='[breath]'),
 Interval(start=62.23376541427553, end=63.41259093575271, label='Oh, poor panda.'),
 Interval(start=70.95091693069224, end=74.0212079176989, label='I think I can check on the bottles. I think that they were fine.'),
 Interval(start=81.48980057193138, end=81.82850536965158, label='Mhm.'),
 Interval(start=86.737566130

In [20]:
import re
re.match("^\[([^\]]+)\]$", "[breath]")

<re.Match object; span=(0, 8), match='[breath]'>

In [22]:
noise_event("[breath]")

True

In [23]:
noise_event("breath")

False